##### lst = [1,2,3];
lst.index(1)

In [6]:
import gym
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential,load_model
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
# from AgentUtil import *
# execfile('AgentUtil.py')

if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
#     agent = PGAgent(state_size, action_size)


[2017-06-20 09:35:50,430] Making new env: Pong-v0


In [7]:
# AgentName = 'pong_minimal-s5L0'
# agent = PGAgent(state_size, action_size,AgentName=AgentName)
# nb_episode = 1000;
# main(agent) 

# AgentName = 'pong_minimal-s5L0'
# agent = PGAgent(state_size, action_size,AgentName=AgentName)
# main(agent) 

In [8]:


# # AgentName = 'pong_minimal-s5L0'
# # agent = PGAgent(state_size, action_size,AgentName=AgentName)
# # # agent.newlog()
# # nb_episode = 1000;
# # main(agent) 


# # %load_ext line_profiler
# # %lprun -f main main(agent,verbose = 0) 
# AgentName = 'pong_minimal-s5L1_i2'
# agent = PGAgent(state_size, action_size,AgentName=AgentName)
# agent.load('Models/pong_minimal-s5L1_i1.h5')
# agent.save()
# agent.newlog()

In [9]:
import gym
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential,load_model
from keras.layers import Dense, Reshape, Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    if 1:
        for i in range(0, len(l), n):
    #         print(l.shape)
    #         print(l[i:i + n,:].shape)
    #         print('yield l[%d:%d + %d,:]'%(i,i,n))
            yield l[i:i + n]

class PGAgent:
    def __init__(self, state_size, action_size,AgentName = 'tst'):
#         self.AgentName = 'pong_minimal-s5L1b-tst'
        self.name = AgentName
        self.AgentFile = 'Models/%s.h5'%AgentName;
        self.LogName = 'Models/%s.log'%AgentName;
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.states = []
        self.gradients = []
        self.rewards = []
        self.probs = []
        self.model = self._build_model()
        self.summary = self.model.summary;

    def _build_model(self):
        model = Sequential()
        model.add(Reshape((80, 80, 1), input_shape=(self.state_size,)))
        model.add(Convolution2D(25, (6, 6), subsample=(3, 3), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Convolution2D(5, (6, 6), subsample=(1, 1), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Convolution2D(5, (6, 6), subsample=(1, 1), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Convolution2D(5, (6, 6), subsample=(1, 1), border_mode='same',
                                activation='relu', init='he_uniform'))
        model.add(Flatten())
#         model.add(Dense(20, activation='relu', init='he_uniform'))
#         model.add(Dense(20, activation='relu', init='he_uniform'))
        model.add(Dense(self.action_size, activation='softmax'))
        opt = Adam(lr=self.learning_rate)
        model.compile(loss='categorical_crossentropy', optimizer=opt)
        return model

    def remember(self, state, action, prob, reward):
        y = np.zeros([self.action_size])
        y[action] = 1
        self.gradients.append(np.array(y).astype('float32') - prob)
        self.states.append(state)
        self.rewards.append(reward)

    def act(self, state):
        # state = state.reshape([1, state.shape[0]])
        aprob = self.model.predict(state, batch_size=1).flatten()
        self.probs.append(aprob)
        prob = aprob / np.sum(aprob)
        action = np.random.choice(self.action_size, 1, p=prob)[0]
        return action, prob

    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, rewards.size)):
            if rewards[t] != 0:
                running_add = 0
            running_add = running_add * self.gamma + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    def train(self,rewards,batch_size = 2000,verbose = 0):
        gradients = np.vstack(self.gradients)
        # rewards = np.vstack(self.rewards)
        # rewards = self.discount_rewards(rewards)
        rewards = (rewards - np.mean(rewards,keepdims=1)) / np.std(rewards)
        gradients *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = self.probs + self.learning_rate * np.squeeze(np.vstack([gradients]))
#         print(X.shape)
        self.model.train_on_batch(X, Y)
#         self.model.train_on_batch(X[0:1000], Y[0:1000])
    

    
# #         X = np.expand_dims(X,axis = 1)
# #         Y = np.expand_dims(Y,axis = 1)
    
# #         gen = ((X[i],Y[i]) for i in range(len(X)));
# #         zipped = zip(X,Y)
#         Xgen = chunks(X,batch_size);
#         Ygen = chunks(Y,batch_size);
#         gen = (x for x in zip(Xgen,Ygen))
# #         gen1 = ((x,y) for x,y in zip(chunks(X,batch_size).next(),chunks(Y,batch_size).next()))
# #         print(gen1.next()[1].shape)
# #         print(chunks(X,batch_size).next().shape)
# #         print()
#         bmax = max(X.shape[0]//batch_size,1)
# #         print(gen)
# #         print(bmax)
# #         try:
# #             self.model.fit_generator(gen, steps_per_epoch = bmax, epochs = 1,verbose = verbose, max_q_size=1)
# #         except StopIteration:
# #             pass
        
# #         bmax = max(X.shape[0]//batch_size,1);
#         for b in range(bmax):
#             self.model.train_on_batch(X[b*batch_size:(b+1)*batch_size,:], Y[b*batch_size:(b+1)*batch_size,:])
# #         self.model.fit(X,Y,  epochs = 1,verbose = verbose)
    
        self.states, self.probs, self.gradients, self.rewards = [], [], [], []

    def load(self, fname=None):
        if not fname:
            fname = self.AgentFile;
        global episode
        self.model = load_model(fname);
        self.summary = self.model.summary;
        
    def readlog(self, LogName=None):
        if not LogName:
            LogName = self.LogName;
        with open(LogName,'rb') as f:
                first = f.readline()      # Read the first line.
                if not first.rstrip('\n'):
#                     print('nothing!')
                    self.episode = 0;
                else:
                    f.seek(-2, 2)             # Jump to the second last byte.
                    while f.read(1) != b"\n": # Until EOL is found...
                        f.seek(-2, 1)         # ...jump back the read byte plus one more.
                    last = f.readline() 
                    lst = last.split('\t');
                    eind = lst.index('Episode')+1;
                    self.episode = int(lst[eind]);

    def save(self, fname=None):
        if not fname:
            fname = self.AgentFile;
        self.model.save(fname);
    def writelog(self, msg, LogName = None):
        if not LogName:
            LogName = self.LogName
        with open(LogName,'a+') as LogFile:
            LogFile.write(msg+'\n');
    def newlog(self):
        open(self.LogName,'w').close();



def main(agent, verbose = 1):
    if __name__ == "__main__":
        env = gym.make("Pong-v0")
        state = env.reset()
        prev_x = None
        score = 0
        episode = 0
    if resume:
        agent.load();
        agent.readlog();
        episode = agent.episode;
    while episode < agent.episode + nb_episode:
        if render:
                env.render()

        cur_x = preprocess(state)
        x = cur_x - prev_x if prev_x is not None else np.zeros(state_size)
        prev_x = cur_x

        action, prob = agent.act(np.expand_dims(x,1).T)
        state, reward, done, info = env.step(action)
        score += reward
        agent.remember(x, action, prob, reward)

        if done:
            episode += 1
            if render: break;
            rewards = np.vstack(agent.rewards)
            rewards = agent.discount_rewards(rewards)
            grads = np.vstack(agent.gradients);
    #             pca.fit (grads )
    #             lvar = pca.explained_variance_ratio_[0];
            lvar = 0;
            var_lst = np.var(grads,axis = 0);
            L1_lst = abs(np.mean(grads,axis = 0));
            L1_norm = np.mean(L1_lst)
            all_var = np.mean(var_lst);            

            agent.train(rewards,batch_size = batch_size,verbose = max(verbose-1,0))
            msg = '%s\t%d\t%s\t%f\t%s\t%f\t%s\t%f\t%s\t%f' % ('Episode',episode,'Score', score,'largest_variance',lvar,'all_var',all_var,'L1_norm',L1_norm);
            if verbose:
                print(msg);
            agent.writelog(msg)

            score = 0
            state = env.reset()
            prev_x = None
            if episode > 1 and episode % 10 == 0 and not render:
                agent.save()
    print('Agent: %s finshed trainning episode %d to %d'%(agent.name, agent.episode,episode))
            
def preprocess(I):
    I = I[35:195]
    I = I[::2, ::2, 0]
    I[I == 144] = 0
    I[I == 109] = 0
    I[I != 0] = 1
    return I.astype(np.float).ravel()



# agent.summary();

In [10]:
if __name__ == "__main__":
    env = gym.make("Pong-v0")
    state = env.reset()
    prev_x = None
    score = 0
    episode = 0

    state_size = 80 * 80
    action_size = env.action_space.n
    
#     agent.summary();
AgentName = 'pong_minimal-s5L1b-tst'
# AgentFile = 'Models/%s.h5'%AgentName;
# LogName = 'Models/%s.log'%AgentName;
# LogFile = open(LogName,'a');
resume = 1;
render = 0;
usegpu = 0;
nb_episode = 5;
batch_size = 1000;

K.clear_session()
if not usegpu:
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, device_count = {'GPU': 0}))
    K.set_session(sess)
else:
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.per_process_gpu_memory_fraction = 0.65
    config.gpu_options.allow_growth=False
    sess = tf.Session(config=config)
    K.set_session(sess)
# agent = PGAgent(state_size, action_size,AgentName=AgentName)



[2017-06-20 09:35:51,651] Making new env: Pong-v0


In [11]:
# AgentName = 'pong_minimal-s5L3'
# agent = PGAgent(state_size, action_size,AgentName=AgentName)
# # agent.newlog()
# nb_episode = 1000;
# main(agent) 


# %load_ext line_profiler
# %lprun -f main main(agent,verbose = 0) 

In [12]:


AgentName = 'pong_minimal-s5L1_i2'
agent = PGAgent(state_size, action_size,AgentName=AgentName)
# agent.newlog()
batch_size = 10;

nb_episode = 2000;
# main(agent) 


%load_ext line_profiler
%lprun -f main main(agent,verbose = 1) 

/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(3, 3))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
/home/shouldsee/.local/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (6, 6), kernel_initializer="he_uniform", activation="relu", padding="same", strides=(1, 1))`
[2017-06-20 09:35:52,415] Making new en

Episode	2387	Score	-16.000000	largest_variance	0.000000	all_var	0.110960	L1_norm	0.004841
Episode	2388	Score	-18.000000	largest_variance	0.000000	all_var	0.108472	L1_norm	0.007432
Episode	2389	Score	-16.000000	largest_variance	0.000000	all_var	0.109838	L1_norm	0.010201
Episode	2390	Score	-15.000000	largest_variance	0.000000	all_var	0.111013	L1_norm	0.003289
Episode	2391	Score	-18.000000	largest_variance	0.000000	all_var	0.109345	L1_norm	0.007803
Episode	2392	Score	-15.000000	largest_variance	0.000000	all_var	0.109099	L1_norm	0.004619
Episode	2393	Score	-18.000000	largest_variance	0.000000	all_var	0.108637	L1_norm	0.005505
Episode	2394	Score	-14.000000	largest_variance	0.000000	all_var	0.109385	L1_norm	0.003016
Episode	2395	Score	-17.000000	largest_variance	0.000000	all_var	0.109992	L1_norm	0.002772
Episode	2396	Score	-16.000000	largest_variance	0.000000	all_var	0.109225	L1_norm	0.007081
Episode	2397	Score	-20.000000	largest_variance	0.000000	all_var	0.108376	L1_norm	0.005256
Episode	23

Episode	2479	Score	-14.000000	largest_variance	0.000000	all_var	0.077608	L1_norm	0.003970
Episode	2480	Score	-14.000000	largest_variance	0.000000	all_var	0.078741	L1_norm	0.004632
Episode	2481	Score	-18.000000	largest_variance	0.000000	all_var	0.073368	L1_norm	0.006415
Episode	2482	Score	-15.000000	largest_variance	0.000000	all_var	0.077517	L1_norm	0.002370
Episode	2483	Score	-17.000000	largest_variance	0.000000	all_var	0.078841	L1_norm	0.004789
Episode	2484	Score	-15.000000	largest_variance	0.000000	all_var	0.079674	L1_norm	0.004453
Episode	2485	Score	-14.000000	largest_variance	0.000000	all_var	0.081334	L1_norm	0.004475
Episode	2486	Score	-11.000000	largest_variance	0.000000	all_var	0.081493	L1_norm	0.005632
Episode	2487	Score	-14.000000	largest_variance	0.000000	all_var	0.077179	L1_norm	0.002641
Episode	2488	Score	-16.000000	largest_variance	0.000000	all_var	0.082654	L1_norm	0.003018
Episode	2489	Score	-15.000000	largest_variance	0.000000	all_var	0.078444	L1_norm	0.002146
Episode	24

Episode	2571	Score	-15.000000	largest_variance	0.000000	all_var	0.065742	L1_norm	0.002525
Episode	2572	Score	-12.000000	largest_variance	0.000000	all_var	0.068769	L1_norm	0.002346
Episode	2573	Score	-17.000000	largest_variance	0.000000	all_var	0.066551	L1_norm	0.002973
Episode	2574	Score	-14.000000	largest_variance	0.000000	all_var	0.066556	L1_norm	0.003730
Episode	2575	Score	-16.000000	largest_variance	0.000000	all_var	0.065651	L1_norm	0.002907
Episode	2576	Score	-18.000000	largest_variance	0.000000	all_var	0.067000	L1_norm	0.004007
Episode	2577	Score	-9.000000	largest_variance	0.000000	all_var	0.067763	L1_norm	0.001207
Episode	2578	Score	-18.000000	largest_variance	0.000000	all_var	0.061721	L1_norm	0.004418
Episode	2579	Score	-10.000000	largest_variance	0.000000	all_var	0.066346	L1_norm	0.003022
Episode	2580	Score	-13.000000	largest_variance	0.000000	all_var	0.067119	L1_norm	0.003705
Episode	2581	Score	-9.000000	largest_variance	0.000000	all_var	0.067184	L1_norm	0.004252
Episode	2582

Episode	2663	Score	-11.000000	largest_variance	0.000000	all_var	0.057801	L1_norm	0.001181
Episode	2664	Score	-11.000000	largest_variance	0.000000	all_var	0.060821	L1_norm	0.004049
Episode	2665	Score	-13.000000	largest_variance	0.000000	all_var	0.055057	L1_norm	0.002170
Episode	2666	Score	-13.000000	largest_variance	0.000000	all_var	0.057393	L1_norm	0.004211
Episode	2667	Score	-8.000000	largest_variance	0.000000	all_var	0.058342	L1_norm	0.004902
Episode	2668	Score	-5.000000	largest_variance	0.000000	all_var	0.056213	L1_norm	0.003348
Episode	2669	Score	-12.000000	largest_variance	0.000000	all_var	0.058061	L1_norm	0.003434
Episode	2670	Score	-10.000000	largest_variance	0.000000	all_var	0.054233	L1_norm	0.002918
Episode	2671	Score	-12.000000	largest_variance	0.000000	all_var	0.058747	L1_norm	0.002949
Episode	2672	Score	-10.000000	largest_variance	0.000000	all_var	0.059231	L1_norm	0.003083
Episode	2673	Score	-13.000000	largest_variance	0.000000	all_var	0.053985	L1_norm	0.004158
Episode	2674

Episode	2755	Score	-12.000000	largest_variance	0.000000	all_var	0.051962	L1_norm	0.003813
Episode	2756	Score	-5.000000	largest_variance	0.000000	all_var	0.052173	L1_norm	0.002962
Episode	2757	Score	-11.000000	largest_variance	0.000000	all_var	0.054754	L1_norm	0.004115
Episode	2758	Score	-13.000000	largest_variance	0.000000	all_var	0.054856	L1_norm	0.001514
Episode	2759	Score	-16.000000	largest_variance	0.000000	all_var	0.055099	L1_norm	0.003782
Episode	2760	Score	-5.000000	largest_variance	0.000000	all_var	0.051484	L1_norm	0.003813
Episode	2761	Score	-8.000000	largest_variance	0.000000	all_var	0.049137	L1_norm	0.002775
Episode	2762	Score	-9.000000	largest_variance	0.000000	all_var	0.051574	L1_norm	0.002860
Episode	2763	Score	-14.000000	largest_variance	0.000000	all_var	0.052913	L1_norm	0.003268
Episode	2764	Score	-11.000000	largest_variance	0.000000	all_var	0.051146	L1_norm	0.002092
Episode	2765	Score	-17.000000	largest_variance	0.000000	all_var	0.052014	L1_norm	0.004194
Episode	2766	S

In [13]:
# # # len(agent.probs)
# # from sklearn.decomposition import PCA
# # X = np.vstack(agent.gradients)
# # pca = PCA(n_components = 1)
# # pca.fit(X)
# # print(pca.explained_variance_ratio_)
# # test
# # for i in range(100):
# env.reset()
# (state,_,_,_)=env.step(1)
# prev_x = state;
# (state,_,_,_)=env.step(2)
# curr_x = state;
# x = preprocess(curr_x-prev_x)
# %lprun -f  agent.act action, prob = agent.act(np.expand_dims(x,1).T)